In [1]:
import pandas as pd
import nltk

### Download Amazon Product Reviews from Kaggle

You can refer helper notebook [Here](https://github.com/suvigyajain0101/Utils/blob/main/DownloadKaggleData.ipynb)

In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [3]:
import contextlib
from google.colab import files
import io

with contextlib.redirect_stdout(io.StringIO()):
    files.upload()

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d datafiniti/consumer-reviews-of-amazon-products

 55% 9.00M/16.3M [00:00<00:00, 86.9MB/s]
100% 16.3M/16.3M [00:00<00:00, 126MB/s] 


In [8]:
! unzip /content/consumer-reviews-of-amazon-products.zip

Archive:  /content/consumer-reviews-of-amazon-products.zip
  inflating: 1429_1.csv              
  inflating: Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv  
  inflating: Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv  


### EDA

Once data is downloaded, we can do a quick EDA and see what exactly we are dealing with!

In [9]:
# Read reviews dataset
reviews_data = pd.read_csv('/content/1429_1.csv')

reviews_data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


We are interested in reviews.text field for Sentiment Analysis

In [10]:
# Subset the dataset to relevant columns - Text
reviews = [i for i in reviews_data['reviews.text'].tolist() if i == i]
reviews[0:5]

['This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.',
 'great for beginner or experienced person. Bought as a gift and she loves it',
 'Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already...',
 "I've had my Fire HD 8 two weeks now and I love it. This tablet is a great value.We are Prime Members and that is where this tablet SHINES. I love being able to easily access all of the Prime content as well as movies you can download and watch laterThis has a 1280/800 screen which has some really nice look to it its nice and crisp and very bright infact it is brighter then the ipad pro costing $900 base model. The build on this fire is INSANELY AWESOME running at only 7.7mm thick and the smooth glossy feel on the back it is really amazing to hold its like the futuristic tab in ur hands.",
 'I bought this for my grand daughter 

### Helper Functions

In [11]:
# Notice how create_output method takes another method as an input. Pretty Cool!

from types import MethodType
from collections import Counter

def create_output(list_of_reviews : str, model : MethodType, count_of_reviews: int  = None , ) -> list:
  '''
  Function takes in 3 params :
    list_of_reviews - Reviews in the form of list
    count_of_reviews - Number of reviews to be processed. Can be used to save time during testing
    model - Model method to be used for Sentiment Analysis

  Returns a list with every element made of 2 elements - Review and it's sentiment
  '''
  output = []

  if count_of_reviews:
    counter = count_of_reviews
  else:
    counter = len(list_of_reviews)

  for i in range(counter):
    output.append([list_of_reviews[i], model(list_of_reviews[i])])


  return output

def compare_output(o1, o2):
  '''
  Function takes in 2 sentiment analysis outputs [[review, sentiment]] and returns following stats - 
  1. Individual value_counts of the sentiments
  2. List of all the elements where sentiments didn't match

  '''
  # Value Counts for output 1
  sentiments_o1 = [i[1] for i in o1]
  value_counts_o1 = Counter(sentiments_o1)

  # Value Counts for output 2
  sentiments_o2 = [i[1] for i in o2]
  value_counts_o2 = Counter(sentiments_o2)

  # Get examples where outputs mismatched
  ################## WIP########################



  return value_counts_o1, value_counts_o2

### NLTK 

Firstly, let's try to use NLTK's Sentiment Analysis Model out-of-the-box

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


True

#### Quick note on NLTK Scoring

As per the [documentation](https://github.com/cjhutto/vaderSentiment#about-the-scoring), 'compound' is the important parameter to judge if the final score is Positive, Negative or Neutral.

The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

* positive sentiment: compound score >= 0.05
* neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
* negative sentiment: compound score <= -0.05

NOTE: The compound score is the one most commonly used for sentiment analysis by most researchers, including the authors.

In [13]:
sia = SentimentIntensityAnalyzer()

def nltk_sentiment(input_string : str) -> bool:
  '''
  Function takes in a string and returns - 
        a. Sentiment with the highest score
        b. True if the sentiment extracted using NLTK Sentiment Analyser is positive, else False
  '''
  compound_score = sia.polarity_scores(input_string)["compound"]

  if compound_score >= 0.05:
    return 'Positive'
  elif compound_score <= -0.05:
    return 'Negative'
  else:
    return 'Neutral'

In [14]:
create_output(reviews, 5, nltk_sentiment)

TypeError: ignored

### Transfer Learning!

We will use HuggingFace's Base Sentiment Analysis Model for Sentiment Analysis - again, out of the box

In [ ]:
! pip install -q transformers

In [ ]:
from transformers import pipeline

# Loading HuggingFace model takes time, so will initialize it once!
sentiment_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

# The model has a token limitation and errors out if the statement exceeds the character count
max_character_count = 128


def huggingface_sentiment(input_string : str, sentiment_model = sentiment_model, max_character_count = max_character_count) -> str:
  '''
  Function takes in a string and returns standardized Sentiment
  '''
  
  truncated_string = input_string[0:max_character_count]

  label = sentiment_model(truncated_string)[0]['label']

  if label == 'POS':
    return 'Positive'
  elif label == 'NEG':
    return 'Negative'
  elif label == 'NEU':
    return 'Neutral'
  else:
    return 'Invalid Sentiment'

In [ ]:
create_output(reviews, 5, huggingface_sentiment)

### Compare NLTK and HuggingFace

In [ ]:
nltk_output = create_output(reviews, nltk_sentiment)
hf_output = create_output(reviews, huggingface_sentiment)

In [ ]:
compare_output(nltk_output, hf_output)